In [174]:
import numpy as np
from copy import deepcopy
class Mancala:
    def __init__(self, holes=7, stones=7, board=None):
        self.n_holes = holes
        self.n_stones = stones
        self.north_store = self.n_holes
        self.south_store = self.n_holes * 2 + 1
        self.north = 'north'
        self.south = 'south'
        self.first_play = None
        self.reset()

        if board is not None:
            self.board = board

    
    def reset(self):
        self.board = np.full((self.n_holes+1) * 2, self.n_stones)
        self.board[self.n_holes] = 0
        self.board[-1] = 0
    
    def step(mancala, side, hole):
        mancala = deepcopy(mancala)
        pos = mancala.get_board_pos(side, hole)
        stones = mancala.board[pos]
        mancala.board[pos] = 0
        pos += 1
        last_pos = 0
        while stones > 0:
            if not mancala.is_opponent_store(side, pos):
                mancala.board[pos] += 1
                stones -= 1
                if stones == 0:
                    last_pos = pos
            pos = (pos + 1) % len(mancala.board)
        if mancala.is_self_store(side, last_pos):
            return mancala
       
        if mancala.is_self_hole(side, last_pos) and mancala.board[last_pos] == 1 \
            and mancala.board[mancala.get_opponent_pos(last_pos)] != 0:
            scores_stored = mancala.get_self_store(side)
            mancala.board[scores_stored] = mancala.board[scores_stored]+ mancala.board[last_pos] \
                + mancala.board[mancala.get_opponent_pos(last_pos)]
            mancala.board[last_pos] = 0
            mancala.board[mancala.get_opponent_pos(last_pos)] = 0
            
        return mancala
    
    
    
            
    def get_self_store(self,side):
        if side == 'south':
            return self.south_store
        else:
            return self.north_store
    def get_opponent_side(self, side):
        if side == 'north':
            return 'south'
        else:
            return 'north'
    
    def is_self_hole(self, side, pos):
        return side == 'south' and self.north_store < pos < self.south_store \
            or side == 'north' and -1 < pos < self.north_store
    
    def get_opponent_pos(self, pos):
        return 2 * self.n_holes - pos
        
    def is_self_store(self,side, pos):
        return side == 'south' and pos == self.south_store \
            or side == 'north' and pos == self.north_store
    
    
    def is_opponent_store(self, side, pos):
        return side == 'south' and pos == self.north_store \
            or side == 'north' and pos == self.south_store
        
    def get_board_pos(self, side, hole):
        return hole - 1 if side == 'north' else hole + self.n_holes
    
    def has_over_half_stones(self, side):
        return self.board[self.get_store(side)] > 49
    
    def __str__(self):
        formatter = {'int': lambda x: f'{x: >3d}'}
        return '{:2d}  {}\n    {}  {}'.format(
            self.board[self.north_store],
            np.array2string(self.board[0:self.north_store][::-1], formatter=formatter),
            np.array2string(self.board[self.north_store+1:self.south_store], formatter=formatter),
            self.board[self.south_store]
        )

# game = Mancala()
# game.step('south', 1)
# print(game)
# game.step('south', 2)
# print(game)
# game.step('north', 2)
# print(game)
# game = Mancala()
# print(game)
# print(Mancala.step(game,'south', 1))
# print(Mancala.step(Mancala.step(game,'south', 1),'north',2))
# print(Mancala.step(game,'south', 1))


class Mancala_alpha_beta(Mancala):
    def __init__(self, holes=7, stones=7, board=None):
        super().__init__(holes, stones, board)

    def is_max_node(side):
        if side == 'north':
            return False
        else:
            return True

    def get_south_scores(self):
        return self.board[self.south_store]

    def get_north_scores(self):
        return self.board[self.north_store]
    # Assume that we maximize the south side and minimize the north side
    def get_heuristics(self):
        return self.get_south_scores() - self.get_north_scores()

    def is_terminal_node(self):
        return self.has_over_half_stones("north") or \
                self.has_over_half_stones("south")

    def get_board(self):
        return self.board
    def get_store(self, side):
        return self.south_store if side == self.south else self.north_store

    def is_empty_hole(self, hole):
        #print(str(hole))
        return self.board[hole] == 0

    def get_all_possible_moves(self, side):
        move_list = []
        #print(self)
        if side == "north":
            for i in range(self.n_holes):
                if not self.is_empty_hole(i):
                    move_list.append(i)
            return move_list
        else:
            for i in range(self.n_holes):
                if not self.is_empty_hole(i + self.n_holes + 1):
                    move_list.append(i)
            return move_list


    # There we don't take several steps in a row of the same player into account,
    # because the opponent will not let that happen.
    def alpha_beta_pruning(mancala, side, alpha = -99, beta = 99, depth = 6):
        if depth == 0 or mancala.is_terminal_node():
            print("H: " + str(mancala.get_heuristics()))
            print(mancala)
            return None, mancala.get_heuristics()
        if Mancala_alpha_beta.is_max_node(side):
            optimal_value = -99
            optimal_move = None
            print("South:")
            print(mancala)
            print(mancala.get_all_possible_moves(side))
            for move in mancala.get_all_possible_moves(side):
                print("S_move: " + str(move + 1))
                print(Mancala_alpha_beta.step(mancala, side, move + 1))
                _, value = Mancala_alpha_beta.alpha_beta_pruning(Mancala_alpha_beta.step(mancala, side, move + 1),\
                                                mancala.get_opponent_side(side), \
                                                alpha, beta, depth-1)
                if value > optimal_value:
                    optimal_value = value
                    optimal_move = move
                alpha = max(alpha, optimal_move)
                if beta <= alpha:
                    print("pruning")
                    break

            return optimal_move, optimal_value
        else:
            optimal_value = 99
            optimal_move = None
            print("north:")
            print(mancala)
            print(mancala.get_all_possible_moves(side))
            print("---------------------")
            for move in mancala.get_all_possible_moves(side):
                print("N_move: " + str(move + 1))
                print(Mancala_alpha_beta.step(mancala, side, move + 1))
                _, value = Mancala_alpha_beta.alpha_beta_pruning(Mancala_alpha_beta.step(mancala, side, move + 1),\
                                                      mancala.get_opponent_side(side), \
                                                      alpha, beta, depth-1)
                if value < optimal_value:
                    optimal_value = value
                    optimal_move = move
                beta = min(beta, optimal_move)

                if beta <= alpha:
                    print(beta)
                    print(alpha)
                    print("pruning")
                    break
            return optimal_move, optimal_value

In [175]:

move,value = Mancala_alpha_beta.alpha_beta_pruning(Mancala_alpha_beta(),'south')


South:
 0  [  7   7   7   7   7   7   7]
    [  7   7   7   7   7   7   7]  0
[0, 1, 2, 3, 4, 5, 6]
S_move: 1
 0  [  7   7   7   7   7   7   7]
    [  0   8   8   8   8   8   8]  1
north:
 0  [  7   7   7   7   7   7   7]
    [  0   8   8   8   8   8   8]  1
[0, 1, 2, 3, 4, 5, 6]
---------------------
N_move: 1
 1  [  8   8   8   8   8   8   0]
    [  0   8   8   8   8   8   8]  1
South:
 1  [  8   8   8   8   8   8   0]
    [  0   8   8   8   8   8   8]  1
[1, 2, 3, 4, 5, 6]
S_move: 2
 1  [  8   8   8   8   8   9   1]
    [  0   0   9   9   9   9   9]  2
north:
 1  [  8   8   8   8   8   9   1]
    [  0   0   9   9   9   9   9]  2
[0, 1, 2, 3, 4, 5, 6]
---------------------
N_move: 1
 1  [  8   8   8   8   8  10   0]
    [  0   0   9   9   9   9   9]  2
South:
 1  [  8   8   8   8   8  10   0]
    [  0   0   9   9   9   9   9]  2
[2, 3, 4, 5, 6]
S_move: 3
 1  [  8   8   8   9   9  11   1]
    [  0   0   0  10  10  10  10]  3
north:
 1  [  8   8   8   9   9  11   1]
    [  0   0   0  1

H: -2
 3  [  1   9   9   9   1   0   7]
    [  1  12   9   9   9   9   9]  1
N_move: 4
 3  [  1   9   9   0   0   7   7]
    [  1  13  10  10   9   9   9]  1
H: -2
 3  [  1   9   9   0   0   7   7]
    [  1  13  10  10   9   9   9]  1
N_move: 5
 3  [  1   9   0   8   0   7   7]
    [  1  13  10  10  10   9   9]  1
H: -2
 3  [  1   9   0   8   0   7   7]
    [  1  13  10  10  10   9   9]  1
N_move: 6
 3  [  1   0   8   8   0   7   7]
    [  1  13  10  10  10  10   9]  1
H: -2
 3  [  1   0   8   8   0   7   7]
    [  1  13  10  10  10  10   9]  1
pruning
pruning
N_move: 4
 1  [  8   8   8   0   7   7   7]
    [  1   9   9   8   8   8   8]  1
South:
 1  [  8   8   8   0   7   7   7]
    [  1   9   9   8   8   8   8]  1
[0, 1, 2, 3, 4, 5, 6]
S_move: 1
 1  [  8   8   8   0   7   7   7]
    [  0  10   9   8   8   8   8]  1
north:
 1  [  8   8   8   0   7   7   7]
    [  0  10   9   8   8   8   8]  1
[0, 1, 2, 4, 5, 6]
---------------------
N_move: 1
 2  [  9   9   9   1   8   8   0]
    [  0

H: -1
 3  [ 10   2   0   9   9   9   1]
    [  1   1  11  11  11   9   9]  2
N_move: 6
 2  [ 10   0   8   9   9   9   1]
    [  0   0  10  10  10   9   9]  2
H: 0
 2  [ 10   0   8   9   9   9   1]
    [  0   0  10  10  10   9   9]  2
N_move: 7
 3  [  0   1   8   9   9   9   2]
    [  1   1  11  11  11  10  10]  2
H: -1
 3  [  0   1   8   9   9   9   2]
    [  1   1  11  11  11  10  10]  2
pruning
N_move: 2
 2  [  9   1   8   8   8   0   7]
    [  1  10   9   9   9   8   8]  1
South:
 2  [  9   1   8   8   8   0   7]
    [  1  10   9   9   9   8   8]  1
[0, 1, 2, 3, 4, 5, 6]
S_move: 1
 2  [  9   1   8   8   8   0   7]
    [  0  11   9   9   9   8   8]  1
north:
 2  [  9   1   8   8   8   0   7]
    [  0  11   9   9   9   8   8]  1
[0, 2, 3, 4, 5, 6]
---------------------
N_move: 1
 3  [ 10   2   9   9   9   1   0]
    [  0  11   9   9   9   8   8]  1
H: -2
 3  [ 10   2   9   9   9   1   0]
    [  0  11   9   9   9   8   8]  1
N_move: 3
 3  [ 10   2   9   9   0   0   7]
    [  1  12  10 

S_move: 5
 1  [  8   9   9   9   9  11   1]
    [  0   8   1   9   0  10  10]  3
north:
 1  [  8   9   9   9   9  11   1]
    [  0   8   1   9   0  10  10]  3
[0, 1, 2, 3, 4, 5, 6]
---------------------
N_move: 1
 1  [  8   9   9   9   9  12   0]
    [  0   8   1   9   0  10  10]  3
H: 2
 1  [  8   9   9   9   9  12   0]
    [  0   8   1   9   0  10  10]  3
0
1
pruning
S_move: 6
 1  [  9   9   9   9   9  11   1]
    [  0   8   1   9   9   0  10]  3
north:
 1  [  9   9   9   9   9  11   1]
    [  0   8   1   9   9   0  10]  3
[0, 1, 2, 3, 4, 5, 6]
---------------------
N_move: 1
 1  [  9   9   9   9   9  12   0]
    [  0   8   1   9   9   0  10]  3
H: 2
 1  [  9   9   9   9   9  12   0]
    [  0   8   1   9   9   0  10]  3
0
1
pruning
S_move: 7
 1  [  0   9   9   9   9  11   1]
    [  0   8   1   9   9   9   0]  13
north:
 1  [  0   9   9   9   9  11   1]
    [  0   8   1   9   9   9   0]  13
[0, 1, 2, 3, 4, 5]
---------------------
N_move: 1
 1  [  0   9   9   9   9  12   0]
    [  0  

 1  [  8   8   8   9   9  10   0]
    [  0   8   8   8   1   9   9]  2
[1, 2, 3, 4, 5, 6]
S_move: 2
 1  [  8   8   8   9   9  11   1]
    [  0   0   9   9   2  10  10]  3
north:
 1  [  8   8   8   9   9  11   1]
    [  0   0   9   9   2  10  10]  3
[0, 1, 2, 3, 4, 5, 6]
---------------------
N_move: 1
 1  [  8   8   8   9   9  12   0]
    [  0   0   9   9   2  10  10]  3
H: 2
 1  [  8   8   8   9   9  12   0]
    [  0   0   9   9   2  10  10]  3
0
1
pruning
S_move: 3
 1  [  8   8   8   9  10  11   1]
    [  0   8   0   9   2  10  10]  3
north:
 1  [  8   8   8   9  10  11   1]
    [  0   8   0   9   2  10  10]  3
[0, 1, 2, 3, 4, 5, 6]
---------------------
N_move: 1
 1  [  8   8   8   9  10  12   0]
    [  0   8   0   9   2  10  10]  3
H: 2
 1  [  8   8   8   9  10  12   0]
    [  0   8   0   9   2  10  10]  3
0
1
pruning
S_move: 4
 1  [  8   8   8  10  10  11   1]
    [  0   8   8   0   2  10  10]  3
north:
 1  [  8   8   8  10  10  11   1]
    [  0   8   8   0   2  10  10]  3
[0, 1, 

South:
 1  [  9   9  10  10  10  11   0]
    [  8   7   7   7   7   0   0]  2
[0, 1, 2, 3, 4]
S_move: 1
 1  [  9   9  10  10  10  11   1]
    [  0   8   8   8   8   1   1]  3
north:
 1  [  9   9  10  10  10  11   1]
    [  0   8   8   8   8   1   1]  3
[0, 1, 2, 3, 4, 5, 6]
---------------------
N_move: 1
 1  [  9   9  10  10  10  12   0]
    [  0   8   8   8   8   1   1]  3
H: 2
 1  [  9   9  10  10  10  12   0]
    [  0   8   8   8   8   1   1]  3
0
1
pruning
S_move: 2
 1  [  9   9  10  10  10  11   1]
    [  8   0   8   8   8   1   1]  3
north:
 1  [  9   9  10  10  10  11   1]
    [  8   0   8   8   8   1   1]  3
[0, 1, 2, 3, 4, 5, 6]
---------------------
N_move: 1
 1  [  9   9  10  10  10  12   0]
    [  8   0   8   8   8   1   1]  3
H: 2
 1  [  9   9  10  10  10  12   0]
    [  8   0   8   8   8   1   1]  3
0
1
pruning
S_move: 3
 1  [  9   9  10  10  10  12   1]
    [  8   7   0   8   8   1   1]  3
north:
 1  [  9   9  10  10  10  12   1]
    [  8   7   0   8   8   1   1]  3
[0,

In [176]:
print(move)

1


In [177]:
print(value)

12
